In [4]:
# 攝像頭即時人臉識別
# Real-time face recognition

# Author:   coneypo
# Blog:     http://www.cnblogs.com/AdaminXie
# GitHub:   https://github.com/coneypo/Dlib_face_recognition_from_camera

# Created at 2018-05-11
# Updated at 2020-05-29

import dlib         # 人臉處理的庫 Dlib
import numpy as np  # 資料處理的庫 Numpy
import cv2          # 影像處理的庫 OpenCV
import pandas as pd # 資料處理的庫 Pandas
import os
import time
from PIL import Image, ImageDraw, ImageFont
import freetype

# 1. Dlib 正向人臉檢測器
detector = dlib.get_frontal_face_detector()

# 2. Dlib 人臉 landmark 特徵點檢測器
predictor = dlib.shape_predictor('data/data_dlib/shape_predictor_68_face_landmarks.dat')

# 3. Dlib Resnet 人臉識別模型，提取 128D 的特徵向量
face_reco_model = dlib.face_recognition_model_v1("data/data_dlib/dlib_face_recognition_resnet_model_v1.dat")


class Face_Recognizer:
    def __init__(self):
        # 用來存放所有錄入人臉特徵的陣列 / Save the features of faces in the database
        self.features_known_list = []

        # 存儲錄入人臉名字 / Save the name of faces known
        self.name_known_cnt = 0
        self.name_known_list = []

        # 存儲當前攝像頭中捕獲到的所有人臉的座標名字 / Save the positions and names of current faces captured
        self.pos_camera_list = []
        self.name_camera_list = []
        # 存儲當前攝像頭中捕獲到的人臉數
        self.faces_cnt = 0
        # 存儲當前攝像頭中捕獲到的人臉特徵
        self.features_camera_list = []

        # Update FPS
        self.fps = 0
        self.frame_start_time = 0

    # 從 "features_all.csv" 讀取錄入人臉特徵
    def get_face_database(self):
        if os.path.exists("data/features_all.csv"):
            path_features_known_csv = "data/features_all.csv"
            csv_rd = pd.read_csv(path_features_known_csv, header=None)
            # 2. 讀取已知人臉資料 / Print known faces
            for i in range(csv_rd.shape[0]):
                features_someone_arr = []
                for j in range(0, 128):
                    if csv_rd.iloc[i][j] == '':
                        features_someone_arr.append('0')
                    else:
                        features_someone_arr.append(csv_rd.iloc[i][j])
                self.features_known_list.append(features_someone_arr)
                self.name_known_list.append("Person_"+str(i+1))
            self.name_known_cnt = len(self.name_known_list)
            print("Faces in Database：", len(self.features_known_list))
            return 1
        else:
            print('##### Warning #####', '\n')
            print("'features_all.csv' not found!")
            print(
                "Please run 'get_faces_from_camera.py' and 'features_extraction_to_csv.py' before 'face_reco_from_camera.py'",
                '\n')
            print('##### End Warning #####')
            return 0

    # 計算兩個128D向量間的歐式距離 / Compute the e-distance between two 128D features
    @staticmethod
    def return_euclidean_distance(feature_1, feature_2):
        feature_1 = np.array(feature_1)
        feature_2 = np.array(feature_2)
        dist = np.sqrt(np.sum(np.square(feature_1 - feature_2)))
        return dist

    # 更新 FPS / Update FPS of Video stream
    def update_fps(self):
        now = time.time()
        self.frame_time = now - self.frame_start_time
        self.fps = 1.0 / self.frame_time
        self.frame_start_time = now

    def draw_note(self, img_rd):
        font = cv2.FONT_ITALIC

        cv2.putText(img_rd, "Face Recognizer", (20, 40), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "FPS:   " + str(self.fps.__round__(2)), (20, 100), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "Faces: " + str(self.faces_cnt), (20, 140), font, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(img_rd, "Q: Quit", (20, 450), font, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    def draw_name(self, img_rd):
        # 在人臉框下面寫人臉名字 / Write names under rectangle
        #寫入中文 無法使用cv2.putText 使用draw _需解決 _imaging C module error in python PIL
        #font = ImageFont.truetype(face, 30)
        #img = Image.fromarray(cv2.cvtColor(img_rd, cv2.COLOR_BGR2RGB))
        #draw = ImageDraw.Draw(img)
        
        font = cv2.FONT_ITALIC
        img=cv2.cvtColor(img_rd, cv2.COLOR_BGR2RGB)
        for i in range(self.faces_cnt):
            cv2.putText(img, self.name_camera_list[i], self.pos_camera_list[i], font, 0.8, (0, 255, 255), 1, cv2.LINE_AA)
            #draw.text(xy=self.pos_camera_list[i], text=self.name_camera_list[i],font=font)
            img_with_name = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        return img_with_name

    # 修改顯示人名
    def modify_name_camera_list(self):
        # Default known name: person_1, person_2, person_3
        self.name_known_list[0] ='Roger'.encode('utf-8').decode()
        self.name_known_list[1] ='Jinwei'.encode('utf-8').decode()
        self.name_known_list[2] ='Liz'.encode('utf-8').decode()
        # self.name_known_list[3] ='xx'.encode('utf-8').decode()
        # self.name_known_list[4] ='xx'.encode('utf-8').decode()

    # 處理獲取的視頻流，進行人臉識別 / Input video stream and face reco process
    def process(self, stream):
        # 1. 讀取存放所有人臉特徵的 csv
        if self.get_face_database():
            while stream.isOpened():
                flag, img_rd = stream.read()
                faces = detector(img_rd, 0)
                kk = cv2.waitKey(1)
                # 按下 q 鍵退出 / Press 'q' to quit
                if kk == ord('q'):
                    break
                else:
                    self.draw_note(img_rd)
                    self.features_camera_list = []
                    self.faces_cnt = 0
                    self.pos_camera_list = []
                    self.name_camera_list = []

                    # 2. 檢測到人臉 / when face detected
                    if len(faces) != 0:
                        # 3. 獲取當前捕獲到的圖像的所有人臉的特徵，存儲到 self.features_camera_list
                        # 3. Get the features captured and save into self.features_camera_list
                        for i in range(len(faces)):
                            shape = predictor(img_rd, faces[i])
                            self.features_camera_list.append(face_reco_model.compute_face_descriptor(img_rd, shape))

                        # 4. 遍歷捕獲到的圖像中所有的人臉 / Traversal all the faces in the database
                        for k in range(len(faces)):
                            print("##### camera person", k + 1, "#####")
                            # 讓人名跟隨在矩形框的下方
                            # 確定人名的位置座標
                            # 先預設所有人不認識，是 unknown
                            # Set the default names of faces with "unknown"
                            self.name_camera_list.append("unknown")

                            # 每個捕獲人臉的名字座標 / Positions of faces captured
                            self.pos_camera_list.append(tuple(
                                [faces[k].left(), int(faces[k].bottom() + (faces[k].bottom() - faces[k].top()) / 4)]))

                            # 5. 對於某張人臉，遍歷所有存儲的人臉特徵
                            # For every faces detected, compare the faces in the database
                            e_distance_list = []
                            for i in range(len(self.features_known_list)):
                                # 如果 person_X 資料不為空
                                if str(self.features_known_list[i][0]) != '0.0':
                                    print("with person", str(i + 1), "the e distance: ", end='')
                                    e_distance_tmp = self.return_euclidean_distance(self.features_camera_list[k],
                                                                                    self.features_known_list[i])
                                    print(e_distance_tmp)
                                    e_distance_list.append(e_distance_tmp)
                                else:
                                    # 空數據 person_X
                                    e_distance_list.append(999999999)
                            # 6. 尋找出最小的歐式距離匹配 / Find the one with minimum e distance
                            similar_person_num = e_distance_list.index(min(e_distance_list))
                            print("Minimum e distance with person", self.name_known_list[similar_person_num])

                            if min(e_distance_list) < 0.4:
                                self.name_camera_list[k] = self.name_known_list[similar_person_num]
                                print("May be person " + str(self.name_known_list[similar_person_num]))
                            else:
                                print("Unknown person")

                            # 矩形框 / Draw rectangle
                            for kk, d in enumerate(faces):
                                # 繪製矩形框
                                cv2.rectangle(img_rd, tuple([d.left(), d.top()]), tuple([d.right(), d.bottom()]),
                                              (0, 255, 255), 2)
                            print('\n')

                        self.faces_cnt = len(faces)
                        # 7. 在這裡更改顯示的人名 / Modify name if needed
                        self.modify_name_camera_list()
                        # 8. 寫名字 / Draw name
                        # self.draw_name(img_rd)
                        img_with_name = self.draw_name(img_rd)
                    else:
                        img_with_name = img_rd

                print("Faces in camera now:", self.name_camera_list, "\n")

                cv2.imshow("camera", img_with_name)

                # 9. 更新 FPS / Update stream FPS
                self.update_fps()

    # OpenCV 調用攝像頭並進行 process
    def run(self):
        cap = cv2.VideoCapture(0)
        cap.set(3, 480)
        self.process(cap)

        cap.release()
        cv2.destroyAllWindows()


def main():
    Face_Recognizer_con = Face_Recognizer()
    Face_Recognizer_con.run()


if __name__ == '__main__':
    main()


Faces in Database： 3
Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera n

##### camera person 1 #####
with person 1 the e distance: 0.24418018138224457
with person 2 the e distance: 0.3970685490195122
with person 3 the e distance: 0.41049542582740634
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.22628361820482534
with person 2 the e distance: 0.40489963933701106
with person 3 the e distance: 0.4098404301041329
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.22717820855787973
with person 2 the e distance: 0.379749503727961
with person 3 the e distance: 0.4046092148232037
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.22837826312891193
with person 2 the e distance: 0.3896897812263197
with person 3 the e distance: 0.4155343692638521
Minimum e distance 

##### camera person 1 #####
with person 1 the e distance: 0.23590540986628344
with person 2 the e distance: 0.4026725661511478
with person 3 the e distance: 0.4099461739337217
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.23482202770447513
with person 2 the e distance: 0.40932167566227307
with person 3 the e distance: 0.40281846912506997
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.2599336458434523
with person 2 the e distance: 0.40336334980692373
with person 3 the e distance: 0.40927095970917604
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.23640065757880716
with person 2 the e distance: 0.39560726268231255
with person 3 the e distance: 0.4117447851885157
Minimum e distan

##### camera person 1 #####
with person 1 the e distance: 0.23742674649638124
with person 2 the e distance: 0.40845170880033216
with person 3 the e distance: 0.3994471186789866
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.24072444085878006
with person 2 the e distance: 0.4215609814763021
with person 3 the e distance: 0.3977462577965264
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.2336672250289127
with person 2 the e distance: 0.4042688298540113
with person 3 the e distance: 0.40341483061401107
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.24075115178186599
with person 2 the e distance: 0.427984513723478
with person 3 the e distance: 0.40104409222113563
Minimum e distance 

Faces in camera now: [] 

Faces in camera now: [] 

Faces in camera now: [] 

##### camera person 1 #####
with person 1 the e distance: 0.22941386454983603
with person 2 the e distance: 0.4508716297083495
with person 3 the e distance: 0.43121836583658013
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.2582000896108721
with person 2 the e distance: 0.4523667875363522
with person 3 the e distance: 0.4436522504324296
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.300988098315952
with person 2 the e distance: 0.5002367295185518
with person 3 the e distance: 0.46658423599418136
Minimum e distance with person Roger
May be person Roger


Faces in camera now: ['Roger'] 

##### camera person 1 #####
with person 1 the e distance: 0.26900036159034635
with person 2 the e distance: 0.478941011